# Question Answering and Summarization with [T5](https://arxiv.org/pdf/1910.10683.pdf)


Google's T5 is a Sequence to Sequence model that was trained on over 15 different NLP datasets with various problem types, raning from Text Summarization, Question Answering, Translation to various semantical deduction tasks, which enriches T5's ability to map token sequences to semantic vectors which contain more meaning, which T5 leverages to generalize across various tasks and even to never before trained tasks.

On top of this, T5 is trained on the standard Word prediction task, which most transformer based models like BERT, GPT, ELMO have been trained on. This gives T5 general knowledge of real world concepts to additionally enhance its understanding.

With T5 you can answer **general knowledge based questions given no context** and in addition answer **questions on text databases**.      
These questions can be asked in natural human.


## What is a `open book question`? 
You can imagine an `open book` question similar to an examen where you are allowed to bring in text documents or cheat sheets that help you answer questions in an examen. Kinda like bringing a history book to an history examen. 

In `T5's` terms, this means the model is given a `question` and an **additional piece of textual information** or so called `context`.

This enables the `T5` model to answer questions on textual datasets like `medical records`,`newsarticles` , `wiki-databases` , `stories` and `movie scripts` , `product descriptions`, 'legal documents' and many more.



## What is a `closed book question`? 
A `closed book question` is the exact opposite of a `open book question`. In an examen scenario, you are only allowed to use what you have memorized in your brain and nothing else.      
In `T5's` terms this means that T5 can only use it's stored weights to answer a `question` and is given **no aditional context**.        
`T5` was pre-trained on the [C4 dataset](https://commoncrawl.org/) which contains **petabytes  of web crawling data**  collected over the last 8 years, including Wikipedia in every language.


This gives `T5` the broad knowledge of the internet stored in it's weights to answer various `closed book questions` 



<!-- [T5]() -->
![T5 GIF](https://1.bp.blogspot.com/-o4oiOExxq1s/Xk26XPC3haI/AAAAAAAAFU8/NBlvOWB84L0PTYy9TzZBaLf6fwPGJTR0QCLcBGAsYHQ/s1600/image3.gif)


<center><h4><b>MODEL LIST</b></h4>

Below is a list of Text Generation models. You can get detailed information about the models by clicking on the links.

|index|model|lang|
|-----:|:-----|----|
| 1| [t5_active_to_passive_styletransfer](https://nlp.johnsnowlabs.com/2022/05/31/t5_active_to_passive_styletransfer_en_3_0.html)  |en|
| 2| [t5_base](https://nlp.johnsnowlabs.com/2022/05/31/t5_base_en_3_0.html)  |en|
| 3| [t5_base_mediqa_mnli](https://nlp.johnsnowlabs.com/2021/02/19/t5_base_mediqa_mnli_en.html)  |en|
| 4| [t5_formal_to_informal_styletransfer](https://nlp.johnsnowlabs.com/2022/05/31/t5_formal_to_informal_styletransfer_en_3_0.html)  |en|
| 5| [t5_grammar_error_corrector](https://nlp.johnsnowlabs.com/2022/01/12/t5_grammar_error_corrector_en.html)  |en|
| 6| [t5_informal_to_formal_styletransfer](https://nlp.johnsnowlabs.com/2022/05/31/t5_informal_to_formal_styletransfer_en_3_0.html)  |en|
| 7| [t5_passive_to_active_styletransfer](https://nlp.johnsnowlabs.com/2022/05/31/t5_passive_to_active_styletransfer_en_3_0.html)  |en|
| 8| [t5_question_generation_small](https://nlp.johnsnowlabs.com/2022/07/05/t5_question_generation_small_en_3_0.html)  |en|
| 9| [t5_small](https://nlp.johnsnowlabs.com/2022/05/31/t5_small_en_3_0.html)  |en|
| 10| [t5_small_wikiSQL](https://nlp.johnsnowlabs.com/2022/05/31/t5_small_wikiSQL_en_3_0.html)  |en|
| 11| [t5_grammar_error_corrector](https://nlp.johnsnowlabs.com/2022/11/28/t5_grammar_error_corrector_en.html)  |en|

</center>

## Colab Setup

In [ ]:
!pip install -q pyspark==3.3.0 spark-nlp==4.2.4

In [ ]:
import sparknlp

spark = sparknlp.start()

from sparknlp.base import *
from sparknlp.common import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 4.2.4
Apache Spark version: 3.3.0


# Download T5 Model and Create Spark NLP Pipeline

In [ ]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document") 

# Can take in document or sentence columns
t5 = T5Transformer.pretrained(name='t5_base',lang='en')\
    .setInputCols('document')\
    .setOutputCol("T5")\
    .setMaxOutputLength(400)

t5_base download started this may take some time.
Approximate size to download 451.8 MB
[OK!]


# Answering Questions 

## Set the Task to `question`


In [ ]:
# Set the task for questions on T5. Depending to what this is currently set, we get different behaivour
t5.setTask('question')

T5TRANSFORMER_8078c2d39352

## Answer **Closed Book Questions**  
Closed book means that no additional context is given and the model must answer the question with the knowledge stored in it's weights

In [ ]:
# Build pipeline with T5
pipe_components = [documentAssembler,t5]
pipeline = Pipeline().setStages( pipe_components)

# define Data
data = [["Who is president of Nigeria? "],
        ["What is the most common language in India? "],
        ["What is the capital of Germany? "],]
df=spark.createDataFrame(data).toDF('text')

#Predict on text data with T5
model = pipeline.fit(df)
annotated_df = model.transform(df)
annotated_df.select(['text','t5.result']).show(truncate=False)

+-------------------------------------------+------------------+
|text                                       |result            |
+-------------------------------------------+------------------+
|Who is president of Nigeria?               |[Muhammadu Buhari]|
|What is the most common language in India? |[Hindi]           |
|What is the capital of Germany?            |[Berlin]          |
+-------------------------------------------+------------------+



## Answer **Open Book Questions** 
These are questions where we give the model some additional context, that is used to answer the question

In [ ]:
from pyspark.sql import SparkSession

context    = 'context: Peters last week was terrible! He had an accident and broke his leg while skiing!'
question1  = 'question: Why was peters week so bad? ' #
question2  = 'question: How did peter broke his leg? ' 
question3  = 'question: How did peter broke his leg? '
 
data = [[question1+context],[question2+context],[question3+context],]
df=spark.createDataFrame(data).toDF('text')

#Predict on text data with T5
model = pipeline.fit(df)
annotated_df = model.transform(df)
annotated_df.select(['text','t5.result']).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------+
|text                                                                                                                             |result                                             |
+---------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------+
|question: Why was peters week so bad? context: Peters last week was terrible! He had an accident and broke his leg while skiing! |[He had an accident and broke his leg while skiing]|
|question: How did peter broke his leg? context: Peters last week was terrible! He had an accident and broke his leg while skiing!|[skiing]                                           |
|question: How did peter broke his leg? context: Peters last week was terrible! 

In [ ]:
# Ask T5 questions in the context of a News Article
question1 = 'question: Who is Jack ma? '
question2 = 'question: Who is founder of Alibaba Group? '
question3 = 'question: When did Jack Ma re-appear? '
question4 = 'question: How did Alibaba stocks react? '
question5 = 'question: Whom did Jack Ma meet? '
question6 = 'question: Who did Jack Ma hide from? '


# from https://www.bbc.com/news/business-55728338 
news_article_context = """ context:
Alibaba Group founder Jack Ma has made his first appearance since Chinese regulators cracked down on his business empire.
His absence had fuelled speculation over his whereabouts amid increasing official scrutiny of his businesses.
The billionaire met 100 rural teachers in China via a video meeting on Wednesday, according to local government media.
Alibaba shares surged 5% on Hong Kong's stock exchange on the news.
"""

data = [
             [question1+ news_article_context],
             [question2+ news_article_context],
             [question3+ news_article_context],
             [question4+ news_article_context],
             [question5+ news_article_context],
             [question6+ news_article_context]]


df=spark.createDataFrame(data).toDF('text')

#Predict on text data with T5
model = pipeline.fit(df)
annotated_df = model.transform(df)
annotated_df.select(['t5.result']).show(truncate=False)

+-----------------------+
|result                 |
+-----------------------+
|[Alibaba Group founder]|
|[Jack Ma]              |
|[Wednesday]            |
|[surged 5%]            |
|[100 rural teachers]   |
|[Chinese regulators]   |
+-----------------------+



# Summarize documents

In [ ]:
# Set the task for questions on T5
t5.setTask('summarize')

T5TRANSFORMER_8078c2d39352

In [ ]:
# https://www.reuters.com/article/instant-article/idCAKBN2AA2WF

text = """(Reuters) - Mastercard Inc said on Wednesday it was planning to offer support for some cryptocurrencies on its network this year, joining a string of big-ticket firms that have pledged similar support.

The credit-card giant’s announcement comes days after Elon Musk’s Tesla Inc revealed it had purchased $1.5 billion of bitcoin and would soon accept it as a form of payment.

Asset manager BlackRock Inc and payments companies Square and PayPal have also recently backed cryptocurrencies.

Mastercard already offers customers cards that allow people to transact using their cryptocurrencies, although without going through its network.

"Doing this work will create a lot more possibilities for shoppers and merchants, allowing them to transact in an entirely new form of payment. This change may open merchants up to new customers who are already flocking to digital assets," Mastercard said. (mstr.cd/3tLaPZM)

Mastercard specified that not all cryptocurrencies will be supported on its network, adding that many of the hundreds of digital assets in circulation still need to tighten their compliance measures.

Many cryptocurrencies have struggled to win the trust of mainstream investors and the general public due to their speculative nature and potential for money laundering.
"""
data = [[text]]
df=spark.createDataFrame(data).toDF('text')
#Predict on text data with T5
model = pipeline.fit(df)
annotated_df = model.transform(df)
annotated_df.select(['t5.result']).show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                                                                                                                                                                                            |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
v = annotated_df.take(1)
print(f"Original Length {len(v[0].text)}   Summarized Length : {len(v[0].T5[0].result)} ")


Original Length 1284   Summarized Length : 352 


In [ ]:
# Full summarized text
v[0].T5[0].result

'mastercard said on Wednesday it was planning to offer support for some cryptocurrencies on its network this year . the credit-card giant’s announcement comes days after Elon Musk’s Tesla Inc revealed it had purchased $1.5 billion of bitcoin . asset manager blackrock and payments companies Square and PayPal have also recently backed cryptocurrencies .'